In [1]:
%pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [2]:
%pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

## Load Library

In [9]:
import torch
import os
import shutil
from PIL import Image
import numpy as np
import csv
from glob import glob
from tqdm import tqdm
import lpips
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline

In [11]:
# ---------------------------
# Set seeds for reproducibility
# ---------------------------
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [52]:
# ---------------------------
# ControlNet helper functions (Updated for multiple models)
# ---------------------------

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

def clean_huggingface_cache(model_path):
    """Remove unnecessary Hugging Face cache directories and .lock files."""
    for root, dirs, files in os.walk(model_path, topdown=False):
        for name in files:
            if name.endswith(".lock"):
                os.remove(os.path.join(root, name))
        for name in dirs:
            if name.startswith("models--") or name == "temp":
                shutil.rmtree(os.path.join(root, name), ignore_errors=True)

def get_latest_snapshot(model_path):
    """Find and move the correct snapshot folder for a downloaded model."""
    if os.path.exists(model_path):
        for subdir in os.listdir(model_path):
            snapshot_path = os.path.join(model_path, subdir, "snapshots")
            if os.path.exists(snapshot_path):
                snapshots = sorted(os.listdir(snapshot_path), reverse=True)
                if snapshots:
                    latest_snapshot = os.path.join(snapshot_path, snapshots[0])
                    for file_name in os.listdir(latest_snapshot):
                        src = os.path.join(latest_snapshot, file_name)
                        dest = os.path.join(model_path, file_name)
                        if not os.path.exists(dest):
                            shutil.move(src, dest)
                    shutil.rmtree(os.path.dirname(latest_snapshot), ignore_errors=True)
                    return model_path
    return model_path

def check_and_download_model(model_name, model_path, is_controlnet=False, model_type=""):
    """Check if the model exists; if not, download and move it to the correct directory."""
    if is_controlnet:
        model_path = os.path.join(model_path, "controlnet", model_type)
    else:
        model_path = os.path.join(model_path, "stable-diffusion")

    if os.path.exists(model_path) and os.listdir(model_path):
        return

    # Download silently
    temp_dir = os.path.join("models", "temp")
    if is_controlnet:
        ControlNetModel.from_pretrained(model_name, cache_dir=temp_dir)
    else:
        StableDiffusionInpaintPipeline.from_pretrained(model_name, cache_dir=temp_dir)

    correct_model_path = get_latest_snapshot(temp_dir)
    os.makedirs(model_path, exist_ok=True)
    for file_name in os.listdir(correct_model_path):
        src = os.path.join(correct_model_path, file_name)
        dest = os.path.join(model_path, file_name)
        if not os.path.exists(dest):
            shutil.move(src, dest)
    shutil.rmtree(temp_dir, ignore_errors=True)

    # ---------------------------
# Updated ControlNet loading with better error handling
# ---------------------------
from huggingface_hub import login

def load_controlnet_models():
    """Load three different ControlNet models for inpainting evaluation."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    models_dir = "models"
    os.makedirs(models_dir, exist_ok=True)

    # First try to login to Hugging Face (optional)
    try:
        login(token="hf_YLBEIQWasZiRkWjteKqsSvEjQwtKwlYNoie")  # Only needed if you hit rate limits
    except:
        pass

    # Download base stable diffusion model
    try:
        check_and_download_model("stabilityai/stable-diffusion-2-inpainting",
                               models_dir,
                               is_controlnet=False)
    except Exception as e:
        print(f"Error downloading base model: {e}")

    # Updated ControlNet models list with verified names
    controlnet_models = {
        "v11p": "lllyasviel/control_v11p_sd15_inpaint",
        "v11f1p": "lllyasviel/control_v11f1e_sd15_ip2p",
        "v11canny": "lllyasviel/control_v11p_sd15_canny"
    }

    controlnet_pipes = {}

    for model_type, model_name in controlnet_models.items():
        try:
            print(f"Downloading {model_name}...")
            check_and_download_model(model_name, models_dir,
                                   is_controlnet=True,
                                   model_type=model_type)

            # Load the ControlNet model
            controlnet_dir = os.path.join(models_dir, "controlnet", model_type)
            controlnet = ControlNetModel.from_pretrained(
                controlnet_dir,
                torch_dtype=torch_dtype,
                local_files_only=True
            ).to(device, dtype=torch_dtype)

            # Create pipeline
            controlnet_pipes[model_type] = StableDiffusionControlNetInpaintPipeline(
                vae=pipe.vae,
                text_encoder=pipe.text_encoder,
                tokenizer=pipe.tokenizer,
                unet=pipe.unet,
                scheduler=pipe.scheduler,
                safety_checker=pipe.safety_checker,
                feature_extractor=pipe.feature_extractor,
                controlnet=controlnet
            ).to(device, dtype=torch_dtype)

        except Exception as e:
            print(f"Failed to load {model_name}: {e}")
            continue

    return controlnet_pipes

def make_divisible_by_8(size):
    """Ensure both width and height are divisible by 8."""
    width, height = size
    width = (width // 8) * 8
    height = (height // 8) * 8
    return width, height

def run_controlnet_inpaint(image_path, mask_path, pipe, reference_images, prompt, output_path, seed=42):
    # Open image and mask
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    original_size = image.size
    adjusted_size = make_divisible_by_8(original_size)

    conditioning = None
    if reference_images:
        conditioning = [
            img.resize(adjusted_size, Image.Resampling.LANCZOS)
            for img in reference_images
        ]

    # Create a generator with a fixed seed for reproducibility
    device = "cuda" if torch.cuda.is_available() else "cpu"
    generator = torch.Generator(device=device).manual_seed(seed)

    result = pipe(
        prompt=prompt,
        image=image.resize(adjusted_size, Image.Resampling.LANCZOS),
        mask_image=mask.resize(adjusted_size, Image.Resampling.LANCZOS),
        controlnet_conditioning_image=conditioning,
        height=adjusted_size[1],
        width=adjusted_size[0],
        generator=generator
    ).images[0]
    result = result.resize(original_size, Image.Resampling.LANCZOS)
    result.save(output_path)


## LPIPS model loading

In [53]:
# ---------------------------
# LPIPS model loading
# ---------------------------
def load_lpips_model(model_dir="models/lpips"):
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, "lpips_alex.pth")
    model = lpips.LPIPS(net='alex')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
    else:
        torch.save(model.state_dict(), model_path)
    model.eval()
    if torch.cuda.is_available():
        model.cuda()
    return model

lpips_model = load_lpips_model()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


## Evaluation functions

In [54]:
# ---------------------------
# Evaluation functions
# ---------------------------
def prepare_for_lpips(pil_image):
    tensor = transforms.ToTensor()(pil_image).unsqueeze(0)
    tensor = tensor * 2 - 1
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

def evaluate_metrics(gt_img, inpaint_img):
    gt_np = np.array(gt_img).astype(np.float32) / 255.0
    inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    if gt_np.shape != inpaint_np.shape:
        inpaint_img = inpaint_img.resize(gt_img.size, Image.Resampling.LANCZOS)
        inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    psnr = compute_psnr(gt_np, inpaint_np, data_range=1.0)

    min_size = min(gt_np.shape[0], gt_np.shape[1])
    win_size = 7 if min_size >= 7 else (min_size if min_size % 2 == 1 else min_size - 1)
    ssim = compute_ssim(gt_np, inpaint_np, win_size=win_size, channel_axis=2, data_range=1.0)

    gt_tensor = prepare_for_lpips(gt_img)
    inpaint_tensor = prepare_for_lpips(inpaint_img)
    with torch.no_grad():
        lpips_distance = lpips_model(gt_tensor, inpaint_tensor).item()

    return psnr, ssim, lpips_distance

## Main combined evaluation

In [55]:
from google.colab import drive
import os
from glob import glob

In [57]:
# ---------------------------
# Main combined evaluation (Updated for your dataset structure)
# ---------------------------
if __name__ == "__main__":
    # For Google Colab - mount your Google Drive
    drive.mount('/content/drive')

   # Set your dataset paths (modify these according to your dataset location)
    base_dir = "/content/drive/MyDrive/dataset_controller/z31"  # Base directory from your screenshot
    image_dir = os.path.join(base_dir, "00079_orig")       # Directory for original images
    mask_dir = os.path.join(base_dir, "00079_fake_mask")                 # Directory for masks
    results_dir = os.path.join(base_dir, "inpainting_results")  # Results directory

    # Create output directories for each ControlNet model
    output_dirs = {
        "v11p": os.path.join(results_dir, "controlnet_v11p"),
        "v11f1p": os.path.join(results_dir, "controlnet_v11f1p"),
        "v11canny": os.path.join(results_dir, "controlnet_v11canny")
    }

    for dir_path in output_dirs.values():
        os.makedirs(dir_path, exist_ok=True)

    # Load all ControlNet models
    controlnet_pipes = load_controlnet_models()

    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []

    # Get all original images (assuming they follow the pattern *_orig.jpg)
    image_paths = sorted(glob(os.path.join(image_dir, "*_orig.jpg")))
    pbar = tqdm(image_paths, total=len(image_paths), desc="Processing images", leave=True)

    for image_path in pbar:
        filename = os.path.basename(image_path)
        # Extract base ID (e.g., "00079" from "00079_orig.jpg")
        base_id = filename.split('_')[0]

        # Find corresponding mask (assuming pattern *_fake_mask.png)
        mask_path = os.path.join(mask_dir, f"{base_id}_fake_mask.png")

        if not os.path.exists(mask_path):
            print(f"Mask not found for {filename}")
            continue  # skip missing masks

        # Process with each ControlNet model
        metrics = {'filename': filename, 'base_id': base_id}
        reference_images = None  # You can add reference images if needed

        for model_type, pipe in controlnet_pipes.items():
            output_filename = f"{base_id}_result_{model_type}.jpg"
            output_path = os.path.join(output_dirs[model_type], output_filename)

            try:
                run_controlnet_inpaint(
                    image_path, mask_path, pipe,
                    reference_images, prompt, output_path, seed=42
                )

                # Evaluate results against the original image
                gt_image = Image.open(image_path).convert("RGB")
                result_image = Image.open(output_path).convert("RGB")

                psnr, ssim, lpips = evaluate_metrics(gt_image, result_image)

                # Store metrics
                metrics[f'{model_type}_PSNR'] = psnr
                metrics[f'{model_type}_SSIM'] = ssim
                metrics[f'{model_type}_LPIPS'] = lpips

            except Exception as e:
                print(f"Error with {model_type} for {filename}: {str(e)}")
                metrics[f'{model_type}_PSNR'] = -1
                metrics[f'{model_type}_SSIM'] = -1
                metrics[f'{model_type}_LPIPS'] = -1
                continue

        evaluation_results.append(metrics)

        # Update progress bar with latest metrics
        latest_metrics = {k: v for k, v in metrics.items() if k not in ['filename', 'base_id']}
        pbar.set_postfix(latest_metrics)

            # Save results to CSV
    csv_file_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        fieldnames = ['filename', 'base_id']
        # Dynamically get all metric fields from first result
        fieldnames.extend([k for k in evaluation_results[0].keys() if k not in ['filename', 'base_id']])

        with open(csv_file_path, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            for row in evaluation_results:
                writer.writerow(row)

    print(f"Processing completed. Results saved to {csv_file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Failed to load lllyasviel/control_v11p_sd15_inpaint: Error no file named config.json found in directory models/controlnet/v11p.
Failed to load lllyasviel/control_v11f1e_sd15_ip2p: lllyasviel/control_v11f1e_sd15_ip2p is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `token` or log in with `huggingface-cli login`.
Failed to load lllyasviel/control_v11p_sd15_canny: Error no file named config.json found in directory models/controlnet/v11canny.


Processing images: 0it [00:00, ?it/s]

Processing completed. Results saved to /content/drive/MyDrive/dataset_controller/z31/inpainting_results/evaluation_results.csv


Processing completed. Results saved to /content/drive/MyDrive/dataset_controller/z31/inpainting_results/evaluation_results.csv


## Download stable-diffusion model

In [86]:
import os
import shutil
from google.colab import drive

# Mount Google Drive properly
drive.mount('/content/drive', force_remount=True)

# Set correct paths
drive_path = "/content/drive/MyDrive/models"
sd_path = f"{drive_path}/stable-diffusion"

# Clean previous attempts
if os.path.exists(sd_path):
    shutil.rmtree(sd_path)
os.makedirs(sd_path, exist_ok=True)

Mounted at /content/drive


In [97]:
# Create directory structure
os.makedirs(f"{sd_path}/feature_extractor", exist_ok=True)
os.makedirs(f"{sd_path}/safety_checker", exist_ok=True)
# Create necessary directories
os.makedirs(f"{sd_path}/tokenizer", exist_ok=True)
os.makedirs(f"{sd_path}/text_encoder", exist_ok=True)

# Download ALL required files (including weights)
files_to_download = {
     # Tokenizer files (NEW)
    "tokenizer/merges.txt": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/merges.txt",
    "tokenizer/special_tokens_map.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/special_tokens_map.json",
    "tokenizer/tokenizer_config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/tokenizer_config.json",
    "tokenizer/vocab.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/vocab.json",

    # Text encoder
    "text_encoder/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/text_encoder/config.json",

    # Config files
    "model_index.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/model_index.json",
    "unet/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/unet/config.json",
    "vae/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/vae/config.json",

    # Model weights (critical!)
    "vae/diffusion_pytorch_model.safetensors": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/vae/diffusion_pytorch_model.safetensors",
    "unet/diffusion_pytorch_model.safetensors": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/unet/diffusion_pytorch_model.safetensors",

    # Supporting files
    "scheduler/scheduler_config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/scheduler/scheduler_config.json",
    "feature_extractor/preprocessor_config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/feature_extractor/preprocessor_config.json",
    "safety_checker/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/safety_checker/config.json"
}

print("Downloading complete model setup...")
for file_path, url in files_to_download.items():
    os.makedirs(os.path.dirname(f"{sd_path}/{file_path}"), exist_ok=True)
    !wget -q {url} -O {sd_path}/{file_path}
    print(f"✓ {file_path}")

print("All files downloaded successfully!")

✓ tokenizer/merges.txt
✓ tokenizer/special_tokens_map.json
✓ tokenizer/tokenizer_config.json
✓ tokenizer/vocab.json
✓ text_encoder/config.json
✓ model_index.json
✓ unet/config.json
✓ vae/config.json
✓ vae/diffusion_pytorch_model.safetensors
✓ unet/diffusion_pytorch_model.safetensors
✓ scheduler/scheduler_config.json
✓ feature_extractor/preprocessor_config.json
✓ safety_checker/config.json
All files downloaded successfully!


In [95]:
print("Verifying files...")
!ls -la {sd_path}
!ls -la {sd_path}/feature_extractor
!ls -la {sd_path}/safety_checker

Verifying files...
total 21
drwx------ 2 root root 4096 Mar 26 08:53 feature_extractor
-rw------- 1 root root  544 Mar 26 08:51 model_index.json
drwx------ 2 root root 4096 Mar 26 08:53 safety_checker
drwx------ 2 root root 4096 Mar 26 08:53 scheduler
drwx------ 2 root root 4096 Mar 26 08:53 unet
drwx------ 2 root root 4096 Mar 26 08:52 vae
total 1
-rw------- 1 root root 342 Mar 26 08:53 preprocessor_config.json
total 0
-rw------- 1 root root 0 Mar 26 08:53 config.json


In [101]:
from diffusers import StableDiffusionInpaintPipeline

try:
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        safety_checker=None,  # Disable if needed
        local_files_only=True
    ).to("cuda")
    print("✅ Model fully loaded!")

except Exception as e:
    print(f"❌ Final loading failed: {e}")
    print("\nComplete troubleshooting:")
    print("1. File sizes:")
    !du -sh {model_path}/*
    print("\n2. Alternative: Use cloud model")
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float16
    )

❌ Final loading failed: name 'model_path' is not defined

Complete troubleshooting:
1. File sizes:
du: cannot access '{model_path}/*': No such file or directory

2. Alternative: Use cloud model


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

## Download ConreolNet models

In [ ]:
# Clean previous attempts
if os.path.exists('/content/drive/MyDrive/models'):
    shutil.rmtree('/content/drive/MyDrive/models')

# 2. MANUAL DOWNLOAD FUNCTION
def download_controlnet(model_name, model_dir):
    model_path = f"/content/drive/MyDrive/models/controlnet/{model_name}"
    if not os.path.exists(model_path):
        os.makedirs(model_path, exist_ok=True)

        # Download essential files
        !wget -q https://huggingface.co/lllyasviel/{model_name}/resolve/main/config.json -P {model_path}
        !wget -q https://huggingface.co/lllyasviel/{model_name}/resolve/main/diffusion_pytorch_model.safetensors -P {model_path}

        # Verify download
        if not os.path.exists(f"{model_path}/config.json"):
            raise FileNotFoundError(f"config.json missing for {model_name}")
        if not os.path.exists(f"{model_path}/diffusion_pytorch_model.safetensors"):
            raise FileNotFoundError(f"Model file missing for {model_name}")

        print(f"✓ Successfully downloaded {model_name}")
    return model_path

# 3. DOWNLOAD SPECIFIC MODELS
try:
    # Download ControlNet models (using verified working versions)
    cn_models = {
        "inpaint": "control_v11p_sd15_inpaint",
        "canny": "control_v11p_sd15_canny",
        "seg": "control_v11p_sd15_seg"
    }

    for name, model_id in cn_models.items():
        download_controlnet(model_id, f"/content/drive/MyDrive/models/controlnet/{name}")

except Exception as e:
    print(f"❌ Download failed: {e}")
    raise

✓ Successfully downloaded control_v11p_sd15_inpaint
✓ Successfully downloaded control_v11p_sd15_canny


In [108]:
# 4. VERIFY AND LOAD MODELS
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline
import torch

def load_model(model_class, model_path):
    try:
        return model_class.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            local_files_only=True
        )
    except Exception as e:
        print(f"❌ Failed to load {model_path}: {str(e)[:200]}")
        raise

try:
    # Load base model
    # pipe = load_model(StableDiffusionInpaintPipeline, "/content/drive/MyDrive/models/stable-diffusion")
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float16
    )

    # Load ControlNets
    controlnets = {}
    for name in cn_models.keys():
        model_path = f"/content/drive/MyDrive/models/controlnet/{name}"
        controlnet = load_model(ControlNetModel, model_path)
        controlnets[name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=pipe.safety_checker,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    print("✅ All models loaded successfully!")
    print("Available ControlNets:", list(controlnets.keys()))
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    raise

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the p

✅ All models loaded successfully!
Available ControlNets: ['inpaint', 'canny']


In [ ]:
# ---------------------------
# Main Evaluation Pipeline (Optimized for Google Drive)
# ---------------------------
if __name__ == "__main__":
    # 1. SETUP ENVIRONMENT
    from google.colab import drive
    import os
    from glob import glob
    from tqdm import tqdm
    from PIL import Image
    import csv

    # Mount Google Drive (force remount if needed)
    drive.mount('/content/drive', force_remount=True)

    # 2. CONFIGURE PATHS
    base_dir = "/content/drive/MyDrive/dataset_controller/z31"
    image_dir = os.path.join(base_dir, "original_images")  # Changed to more standard naming
    mask_dir = os.path.join(base_dir, "masks")
    results_dir = os.path.join(base_dir, "inpainting_results")

    # Model paths (update these to match your Google Drive structure)
    model_paths = {
        "v11p": "/content/drive/MyDrive/models/controlnet/inpaint",
        "v11f1p": "/content/drive/MyDrive/models/controlnet/ip2p",
        "v11canny": "/content/drive/MyDrive/models/controlnet/canny"
    }

    # 3. INITIALIZE DIRECTORIES
    os.makedirs(results_dir, exist_ok=True)
    for model_type in model_paths.keys():
        os.makedirs(os.path.join(results_dir, f"controlnet_{model_type}"), exist_ok=True)

    # 4. LOAD MODELS (with error handling)
    try:
        from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline
        import torch

        # Load base model
        pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float16
      )

        # Load ControlNet models
        controlnet_pipes = {}
        for model_type, model_path in model_paths.items():
            try:
                controlnet = ControlNetModel.from_pretrained(
                    model_path,
                    torch_dtype=torch.float16,
                    local_files_only=True
                )

                controlnet_pipes[model_type] = StableDiffusionControlNetInpaintPipeline(
                    vae=pipe.vae,
                    text_encoder=pipe.text_encoder,
                    tokenizer=pipe.tokenizer,
                    unet=pipe.unet,
                    scheduler=pipe.scheduler,
                    safety_checker=None,
                    feature_extractor=pipe.feature_extractor,
                    controlnet=controlnet
                )
                print(f"✓ Loaded {model_type} successfully")
            except Exception as e:
                print(f"⚠️ Failed to load {model_type}: {str(e)[:200]}...")
                continue

    except Exception as e:
        print(f"❌ Critical model loading error: {e}")
        raise

    # 5. EVALUATION PIPELINE
    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []
    image_paths = sorted(glob(os.path.join(image_dir, "*.jpg")))  # Updated pattern
    pbar = tqdm(image_paths, desc="Processing images", dynamic_ncols=True)

    for image_path in pbar:
        filename = os.path.basename(image_path)
        base_id = os.path.splitext(filename)[0].replace("_orig", "")

        # Find mask (supports multiple extensions)
        mask_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            test_path = os.path.join(mask_dir, f"{base_id}_mask{ext}")
            if os.path.exists(test_path):
                mask_path = test_path
                break

        if not mask_path:
            print(f"⚠️ Mask not found for {filename}")
            continue

        metrics = {'filename': filename, 'base_id': base_id}

        for model_type, pipe in controlnet_pipes.items():
            output_path = os.path.join(results_dir, f"controlnet_{model_type}", f"{base_id}_result.jpg")

            try:
                # Run inpainting
                image = Image.open(image_path).convert("RGB")
                mask = Image.open(mask_path).convert("L")

                result = pipe(
                    prompt=prompt,
                    image=image,
                    mask_image=mask,
                    height=image.size[1],
                    width=image.size[0],
                    num_inference_steps=30,
                    generator=torch.Generator("cuda").manual_seed(42)
                ).images[0]

                result.save(output_path)

                # Evaluate
                psnr, ssim, lpips = evaluate_metrics(image, result)
                metrics.update({
                    f'{model_type}_PSNR': psnr,
                    f'{model_type}_SSIM': ssim,
                    f'{model_type}_LPIPS': lpips
                })

            except Exception as e:
                print(f"⚠️ Error processing {model_type} for {filename}: {str(e)[:200]}...")
                metrics.update({
                    f'{model_type}_PSNR': -1,
                    f'{model_type}_SSIM': -1,
                    f'{model_type}_LPIPS': -1
                })

        evaluation_results.append(metrics)
        pbar.set_postfix({k: v for k, v in metrics.items() if k not in ['filename', 'base_id']})

    # 6. SAVE RESULTS
    csv_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        with open(csv_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=evaluation_results[0].keys())
            writer.writeheader()
            writer.writerows(evaluation_results)

    print(f"\n✅ Evaluation complete! Results saved to:\n{csv_path}")
    print(f"Generated {len(evaluation_results)} results")